In [2]:
# rag.py file

from dotenv import load_dotenv
import psycopg2
import os

# Load environment variables
load_dotenv()

# Establish connection to PostgreSQL database using environment variables
conn = psycopg2.connect(
    database=os.getenv("PGDATABASE"),
    user=os.getenv("PGUSER"),
    password=os.getenv("PGPASSWORD"),
    host=os.getenv("PGHOST"),
    port=os.getenv("PGPORT"),
)


# Create a cursor to execute SQL commands
cur = conn.cursor()

In [3]:
try:
    with conn.cursor() as cur:
        cur.execute("SELECT 1")
    print({"status": "healthy", "database": "connected"})
except Exception as e:
    print({"status": "unhealthy", "database": str(e)})


{'status': 'healthy', 'database': 'connected'}


In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector

embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv("SCW_SECRET_KEY"),
    openai_api_base=os.getenv("SCW_GENERATIVE_APIs_ENDPOINT"),
    model="sentence-transformers/sentence-t5-xxl",
    tiktoken_enabled=False,
)

In [6]:
connection_string = f"postgresql+psycopg2://{conn.info.user}:{conn.info.password}@{conn.info.host}:{conn.info.port}/{conn.info.dbname}"
vector_store = PGVector(connection=connection_string, embeddings=embeddings)

In [7]:
# rag.py

import boto3
from langchain_community.document_loaders import S3FileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [8]:
# rag.py

session = boto3.session.Session()
client_s3 = session.client(
    service_name="s3",
    endpoint_url=os.getenv("SCW_BUCKET_ENDPOINT", ""),
    aws_access_key_id=os.getenv("SCW_ACCESS_KEY", ""),
    aws_secret_access_key=os.getenv("SCW_SECRET_KEY", ""),
)
paginator = client_s3.get_paginator("list_objects_v2")
page_iterator = paginator.paginate(Bucket=os.getenv("SCW_BUCKET_NAME", ""))

In [9]:
def create_context_prompt(document_content, chunk_text):
    """
    Creates a well-structured prompt for context generation.
    """

    prompt_template = """Here is the chunk we want to situate within the whole document 
<document>
{document}
</document>

<chunk_to_analyze>
{chunk}
</chunk_to_analyze>
Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else. 
."""

    return prompt_template.format(
        document=document_content.strip(), chunk=chunk_text.strip()
    )

In [30]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_openai import OpenAIEmbeddings
#rag.py

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain
from lunary import LunaryCallbackHandler


doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
    """🎨 Branding resources 
Ultraviolet provides with all the resources, guidelines, elements, and principles that are needed to create assets and communicate around our brand. It ensures consistency in how we present ourselves to the world and supports effective communication across our teams!


  [Ultraviolet](https://ultraviolet.scaleway.com/6dd9b5c45/p/425c81-overview)

Nos valeurs 
Singularité

Nous sommes tous des êtres singuliers, nos équipes le sont aussi. Cette richesse qui fait notre force alimente notre aventure et façonne notre histoire."""
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2


def setup_lunary():
    # Setup Lunary handler
    lunary_handler = LunaryCallbackHandler(app_id=os.getenv("LUNARY_PUBLIC_KEY"))

    return lunary_handler

#rag.py

llm = ChatOpenAI(
        base_url=os.getenv("SCW_GENERATIVE_APIs_ENDPOINT"),
        api_key=os.getenv("SCW_SECRET_KEY"),
        model="llama-3.1-8b-instruct",
        callbacks=[setup_lunary()],
        )

prompt = hub.pull("rlm/rag-prompt")

@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vector_store.similarity_search_with_score(query))
    
    filtered_docs = [doc for doc, score in zip(docs, scores) if score <= 0.2]
    

    # Only store scores for documents that pass the threshold
    for doc in filtered_docs:
        doc.metadata["score"] = doc.metadata.get("score", 0.0)

    return docs


# ensemble_retriever = EnsembleRetriever(
#     retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
# )
ensemble_retriever = retriever



/Users/hugophilipp/Documents/dev/tool-langgraph-llm/.venv/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [31]:
import time

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)


rag_chain_with_source = RunnableParallel(
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)



In [33]:
rag_chain_with_source.invoke("Quelle est le programme du premier jour ?")

{'context': (Document(id='9c5d1ae6-e0f3-470a-8726-30a9463affcd', metadata={'url': 'https://confluence.infra.online.net/pages/viewpage.action?pageId=232395837', 'name': 'Welcome to Scaleway !', 'source': 'arrivee_scw/bienvenue.txt', 'chunk_id': 1, 'position': 996, 'timestamp': '2024-10-25T15:22:38.214492', 'chunk_size': 996, 'score': 0.0}, page_content="# Ton premier jour*\n\n9h00: Petit-déjeuner 10h00: Présentation par l'équipe IT Help Desk (+ configuration des accès) 10h55: Pause 11h00: Présentation de Scaleway par Stéphanie KOLARIC-MARTELLIERE et présentation de la DRH par un membre de l'équipe 12h00: Tour des locaux + déjeuner avec ton manager 14h00: Présentation par l'équipe Product Management, Design et Product Documentation *Cet agenda pourrait avoir des modifications.\n\n# La Direction des Ressources Humaines\n\nL'équipe RH est disponible pour vous accompagner tout au long de votre carrière chez Scaleway. Vous trouverez une présentation rapide de notre département:\n\nScaleway\n

In [22]:

for chunk in rag_chain_with_source.stream("Quelle est le programme du premier  du jour ?"):
    if "answer" in chunk.keys() : 
        print (chunk["answer"], end="")

    if "context" in chunk.keys() : 
        for e in chunk["context"] :
            print ("metadata : ", e.metadata)

    time.sleep(0.02)

(Document(id='9c5d1ae6-e0f3-470a-8726-30a9463affcd', metadata={'url': 'https://confluence.infra.online.net/pages/viewpage.action?pageId=232395837', 'name': 'Welcome to Scaleway !', 'source': 'arrivee_scw/bienvenue.txt', 'chunk_id': 1, 'position': 996, 'timestamp': '2024-10-25T15:22:38.214492', 'chunk_size': 996}, page_content="# Ton premier jour*\n\n9h00: Petit-déjeuner 10h00: Présentation par l'équipe IT Help Desk (+ configuration des accès) 10h55: Pause 11h00: Présentation de Scaleway par Stéphanie KOLARIC-MARTELLIERE et présentation de la DRH par un membre de l'équipe 12h00: Tour des locaux + déjeuner avec ton manager 14h00: Présentation par l'équipe Product Management, Design et Product Documentation *Cet agenda pourrait avoir des modifications.\n\n# La Direction des Ressources Humaines\n\nL'équipe RH est disponible pour vous accompagner tout au long de votre carrière chez Scaleway. Vous trouverez une présentation rapide de notre département:\n\nScaleway\n\nQui sommes\n\nnous?\n\nN

In [15]:
for chunk in rag_chain_with_source.stream("Ca va? Quelle model es tu ?"):
    if "answer" in chunk.keys() : 
        print (chunk["answer"], end="")

    if "context" in chunk.keys() : 
        for e in chunk["context"] :
            print ("metadata : ", e.metadata)

    time.sleep(0.02)

(Document(id='fe18de2e-094a-4754-b888-a0a9bd7baf4b', metadata={'url': 'https://confluence.infra.online.net/pages/viewpage.action?pageId=232395837', 'name': 'Welcome to Scaleway !', 'source': 'arrivee_scw/bienvenue.txt', 'chunk_id': 12, 'position': 11736, 'timestamp': '2024-10-25T15:22:38.214519', 'chunk_size': 978}, page_content="Nom Prénom Pseudo (si tu en as un) Job Team Localisation Numéro de téléphone professionnel Photo En attendant, tu peux d'ores et déjà rejoindre les channels généraux:\n\n>  #announcements, le channel officiel! C’est ici que tu retrouveras toutes les annonces RH, organisationnelles, etc. >  #all-hands, le channel dédié au All-Hands Meeting! >  #ask-anything, si tu as une question générale dont la réponse pourrait intéresser plusieurs collaborateurs. >  #dailywins, le canal dédié à la célébration des réussites quotidiennes! >  #general-board, le canal de communication sur lequel chacun est libre de partager sur des sujets inter-équipes, inter-sites etc... >  #ma

# Metadata fetching 